In [922]:
import math

from pyspark import RDD, SparkContext
from sklearn.cluster import KMeans
from collections import defaultdict
import random
from itertools import combinations

from copy import deepcopy

In [374]:
sc = SparkContext("local", "HW4").getOrCreate()
sc.setLogLevel("WARN")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=HW4, master=local) created by __init__ at <ipython-input-2-0a2feb455107>:1 

In [1560]:
def zero_vector():
    return [0.0 for _ in range(n_dim)]


def maha_dist(point, DS_N, DS_SUM, DS_SUM_SQ):

    distances_to_cluster = {}
    for cluster,_ in DS_N.items():

        d = 0
        N = DS_N[cluster]
        SUM = DS_SUM[cluster]
        SUM_SQ = DS_SUM_SQ[cluster]
        #print(N,SUM, SUM_SQ)

        centroid = [sum/N for sum in SUM]
        sigma =  [ ((SUM_SQ[i]/N) - centroid[i]**2)**0.5 for i in range(len(SUM_SQ))]

        for i in range(len(centroid)):
            d += pow((point[i] - centroid[i]) / sigma[i], 2)

        distances_to_cluster[cluster] = d**0.5


    return(point, distances_to_cluster)

def summarize_results_of_round(round_num):
    num_discard_pts = 0
    for cluster,points in DS_N.items():
        num_discard_pts+=points

    num_cs_clusters = 0
    num_cs_pts = 0
    for cluster,points in CS_N.items():
        num_cs_clusters+=1
        num_cs_pts+=points

    num_rs_points = len(RS_set)
    round_output.append([round_num, num_discard_pts, num_cs_clusters, num_cs_pts, num_rs_points])

def inter_cluster_maha_distance_CS(CS_id1, CS_id2):
    distances_to_cluster = {}
    # CLUSTER 1
    N1 = CS_N[CS_id1]
    SUM1 = CS_SUM[CS_id1]
    SUM_SQ1 = CS_SUM_SQ[CS_id1]

    centroid1 = [sum/N1 for sum in SUM1]
    sigma1 =  [ ((SUM_SQ1[i]/N1) - centroid1[i]**2)**0.5 for i in range(len(SUM_SQ1))]

    #print(centroid1,sigma1)
    #CLUSTER 2
    N2 = CS_N[CS_id2]
    SUM2 = CS_SUM[CS_id2]
    SUM_SQ2 = CS_SUM_SQ[CS_id2]

    centroid2 = [sum/N2 for sum in SUM2]
    sigma2 =  [ ((SUM_SQ2[i]/N2) - centroid2[i]**2)**0.5 for i in range(len(SUM_SQ2))]
    #print(centroid2,sigma2)

    d = 0
    #d2 = 0
    for i in range(len(centroid1)):
        d += pow((centroid1[i] - centroid2[i]) / (sigma1[i] * sigma2[i]), 2)
        #d2 += pow((centroid1[i] - centroid2[i]) / sigma2[i], 2)


    return d**0.5



def merge_CS_cluster(cluster1, cluster2):

    CS_N[cluster1] += CS_N[cluster2]
    CS_SUM[cluster1]=[sum(x) for x in zip(CS_SUM[cluster1], CS_SUM[cluster2])]
    CS_SUM_SQ[cluster1]=[sum(x) for x in zip(CS_SUM_SQ[cluster1], CS_SUM_SQ[cluster2])]
    CS_N.pop(cluster2)
    CS_SUM.pop(cluster2)
    CS_SUM_SQ.pop(cluster2)

def inter_cluster_maha_distance_CS_DS(CS_id1, DS_ID):
    distances_to_cluster = {}
    # CLUSTER 1
    N1 = CS_N[CS_id1]
    SUM1 = CS_SUM[CS_id1]
    SUM_SQ1 = CS_SUM_SQ[CS_id1]
    #print(N1, SUM1,SUM_SQ1)
    centroid1 = [sum/N1 for sum in SUM1]
    sigma1 =  [ ((SUM_SQ1[i]/N1) - centroid1[i]**2)**0.5 for i in range(len(SUM_SQ1))]
    #print(centroid1, sigma1)

    #CLUSTER 2
    N2 = DS_N[DS_ID]
    SUM2 = DS_SUM[DS_ID]
    SUM_SQ2 = DS_SUM_SQ[DS_ID]
    #print(N2, SUM2,SUM_SQ2)
    centroid2 = [sum/N2 for sum in SUM2]
    sigma2 =  [ ((SUM_SQ2[i]/N2) - centroid2[i]**2)**0.5 for i in range(len(SUM_SQ2))]
    #print(centroid2,sigma2)

    d = 0
    #d2 = 0
    for i in range(len(centroid1)):
        d += pow((centroid1[i] - centroid2[i]) / (sigma1[i] * sigma2[i]), 2)
    return d**1/2


def get_cluster_distances(point):
    distance = {}
    for cluster, centroid in DS_Centroids.items():
        c_dist = 0
        for i in range(len(point)):
            c_dist += (point[i] - centroid[i]) ** 2
        distance[cluster] = float(c_dist ** (0.5))

    return (distance)


def assign_cluster(point):
    id = point[0]
    vector = point[1]
    if vector in RS_set:
        return -1
    dist_dict = get_cluster_distances(vector)
    closest = min(dist_dict.items(), key=lambda x: x[1])
    return (closest[0])

In [1634]:
input = "data.txt"

n_cluster = 10
output = "outpu22t.txt"
round_output = []



In [1562]:
inputRDD = sc.textFile(input).map(lambda line: [float(x) for x in line.split(',')])

STEP 1

In [1563]:
random.seed(23)

In [1564]:
shuffled_data = inputRDD.map(lambda x: (x, random.random())).sortBy(lambda x:x[1]).map(lambda x:x[0]).collect()

In [1565]:
subset_size = int(len(shuffled_data)/5)

In [1566]:
splits = [0, subset_size, subset_size*2, subset_size*3, subset_size*4, len(shuffled_data)]

In [1567]:
k_means_input = [i[2:] for i in shuffled_data[splits[0]:splits[1]]]

In [1568]:
n_dim = len(k_means_input[0])
threshold = 2 * (n_dim ** (0.5))

STEP 2

In [1569]:
k_means = KMeans(n_clusters=n_cluster*15).fit(k_means_input)

STEP 3

In [1570]:
RS_dict = defaultdict(int)

In [1571]:
for assignment in k_means.labels_:
    RS_dict[assignment]+=1

In [1572]:
RS_clusters  = [k for k,v in RS_dict.items() if v==1]

In [1573]:
RS_set = []
RS_idx = []

In [1574]:
for idx, cluster in enumerate(k_means.labels_):
    if cluster in RS_clusters:
        RS_idx.append(idx)
        RS_set.append(k_means_input[idx])

In [1575]:
k_means_input_2 = [point for idx,point in enumerate(k_means_input) if idx not in RS_idx]

STEP 4

In [1576]:
k_means = KMeans(n_clusters=n_cluster).fit(k_means_input_2)

STEP 5

In [1577]:
DS_N = defaultdict(int)
DS_SUM = defaultdict(zero_vector)
DS_SUM_SQ = defaultdict(zero_vector)
for idx, cluster in enumerate(k_means.labels_):
    DS_N[cluster]+=1
    #DS_SUM[cluster]=k_means_input_2[idx]
    DS_SUM[cluster]=[sum(x) for x in zip(DS_SUM[cluster], k_means_input_2[idx])]
    DS_SUM_SQ[cluster]=[sum(x) for x in zip(DS_SUM_SQ[cluster], [x**2 for x in k_means_input_2[idx]])]

In [1578]:
DS_SUM

defaultdict(<function __main__.zero_vector()>,
            {4: [978561.7243845094,
              -864129.3929123798,
              932660.1178491778,
              -841067.7083465558,
              -932405.8763028594,
              878818.5391262601,
              -956067.6776580799,
              940398.4401614534,
              -985980.615347457,
              -886602.7681569222],
             7: [616747.1408328012,
              -594004.582612324,
              547227.2018264135,
              582006.7706795882,
              -595627.4490728762,
              -630659.3316867265,
              -575234.4679668192,
              -490086.5542190606,
              -558720.3504178697,
              630792.0430051644],
             2: [762466.7753325464,
              -790400.8500304486,
              -796490.6963693078,
              -743156.8813795155,
              796851.9896124286,
              752142.4571814474,
              810495.2763102924,
              799606.143945496,
      

STEP 6

In [1579]:
try:
    k_means = KMeans(n_clusters=int(len(RS_set)*0.9)).fit(RS_set)
    #k_means = KMeans(n_clusters=n_cluster*2).fit(RS_set)
except(ValueError):
    k_means = KMeans(n_clusters=len(RS_set)).fit(RS_set)

In [1580]:
# Leave RS that has single elements alone
RS_dict_new = defaultdict(int)
#for assignment in k_means.labels_:
for assignment in k_means.labels_:
    RS_dict_new[assignment] += 1

In [1581]:
RS_set_new = []

In [1582]:
RS_clusters_new = [k for k, v in RS_dict_new.items() if v == 1]

In [1583]:
for idx, cluster in enumerate(k_means.labels_):
    if cluster in RS_clusters_new:
        RS_set_new.append(RS_set[idx])

In [1584]:
len(RS_set_new)

12

In [1585]:
CS_set = []

In [1586]:
CS_clusters_new = [k for k, v in RS_dict_new.items() if v != 1]

In [1587]:
CS_clusters_new

[3, 7]

In [1588]:
#for idx, cluster in enumerate(k_means.labels_):
for idx, cluster in enumerate(k_means.labels_):
    if cluster in CS_clusters_new:
        CS_set.append((cluster, RS_set[idx]))

In [1589]:
CS_set

[(3,
  [727.9210742312863,
   855.8532659456439,
   645.3926646449108,
   549.2529438142071,
   882.5475231322887,
   822.3497185455697,
   680.5010772147245,
   941.4391835684437,
   786.365706185587,
   511.36150889816304]),
 (3,
  [633.935278461237,
   827.3916656957927,
   787.731869223562,
   713.225653725925,
   883.2224037489441,
   812.5208518305931,
   742.7999339388975,
   792.9131072654568,
   633.8048379858103,
   596.7692107130416]),
 (7,
  [926.3919413069962,
   636.1659751280349,
   717.0570669250868,
   897.9535339242541,
   788.3146958601612,
   672.7337145809307,
   590.6979270459199,
   896.0154170747182,
   887.4563722204036,
   851.8592279550278]),
 (7,
  [836.864423352014,
   804.4107284103391,
   532.8135000398509,
   760.8771569892986,
   550.8845688899164,
   662.1520455102433,
   739.2301733918056,
   801.4815742066087,
   736.730931435796,
   712.2317369411039])]

In [1591]:
CS_N = defaultdict(int)
CS_SUM = defaultdict(zero_vector)
CS_SUM_SQ = defaultdict(zero_vector)
for cluster, point in CS_set:
    CS_N[cluster]+=1
    CS_SUM[cluster]=[sum(x) for x in zip(CS_SUM[cluster], point)]
    CS_SUM_SQ[cluster]=[sum(x) for x in zip(CS_SUM_SQ[cluster], [x**2 for x in point])]

In [1592]:
CS_N

defaultdict(int, {3: 2, 7: 2})

In [1593]:
RS_set = RS_set_new

In [1594]:
summarize_results_of_round(1)

In [1595]:
round_output

[[1, 64446, 2, 4, 12]]

STEP 7

In [1598]:
for round in range(2,6):
    print(DS_N)
    print(DS_SUM)
    # STEP 7
    next_round_input = sc.parallelize([i[2:] for i in shuffled_data[splits[round-1]:splits[round]]])
    Mahalanobis_distances = next_round_input.map(lambda x: maha_dist(x, DS_N, DS_SUM, DS_SUM_SQ)).collect()

    threshold = 2 * (n_dim ** (1 / 2))
    to_include_in_DS = []
    to_include_in_RS = []

    # STEP 8
    for i in Mahalanobis_distances:
        point = i[0]
        distances = i[1]
        min_dist = min(distances.items(), key = lambda x:x[1])
        closest_cluster = min_dist[0]
        min_dist = min_dist[1]
        RS_flag = True
        if min_dist<threshold:
            to_include_in_DS.append(point)
            DS_N[closest_cluster] += 1
            DS_SUM[closest_cluster] = [sum(x) for x in zip(DS_SUM[closest_cluster], point)]
            DS_SUM_SQ[closest_cluster] = [sum(x) for x in zip(DS_SUM_SQ[closest_cluster], [x ** 2 for x in point])]
        else:
            to_include_in_RS.append(point)

    print("New RS points")


    #STEP 9
    new_RS = []
    for point in to_include_in_RS:
        _, distances = maha_dist(point, CS_N, CS_SUM, CS_SUM_SQ)
        min_dist = min(distances.items(), key = lambda x:x[1])
        print(min_dist)
        closest_cluster = min_dist[0]
        min_dist = min_dist[1]
        #print(min_dist)
        if min_dist<threshold:
            print("move to cs")
            CS_N[closest_cluster]+=1

            CS_SUM[closest_cluster]=[sum(x) for x in zip(CS_SUM[closest_cluster], point)]
            CS_SUM_SQ[closest_cluster]=[sum(x) for x in zip(CS_SUM_SQ[closest_cluster], [x**2 for x in point])]
        else:
            print("still in rs")
            new_RS.append(point)
            #RS_set_new.append(point)

    # Step 10
    RS_set = new_RS + RS_set_new
    try:
        k_means = KMeans(n_clusters=int(len(RS_set) * 0.9)).fit(RS_set)
        #k_means = KMeans(n_clusters=n_cluster*2).fit(RS_set)
    except(ValueError):
        k_means = KMeans(n_clusters=len(RS_set)).fit(RS_set)
    RS_dict_new = defaultdict(int)

    for assignment in k_means.labels_:
        RS_dict_new[assignment] += 1
    RS_set_new = []
    RS_clusters_new = [k for k, v in RS_dict_new.items() if v == 1]
    for idx, cluster in enumerate(k_means.labels_):
        if cluster in RS_clusters_new:
            RS_set_new.append(RS_set[idx])

    CS_set_new = []
    CS_clusters_new = [k for k, v in RS_dict_new.items() if v != 1]

    for idx, cluster in enumerate(k_means.labels_):
        if cluster in CS_clusters_new:
            CS_set_new.append((cluster, RS_set[idx]))
    for cluster, point in CS_set_new:
        cluster = round * 100 + cluster
        CS_N[cluster] += 1
        #DS_SUM[cluster]=k_means_input_2[idx]
        CS_SUM[cluster] = [sum(x) for x in zip(CS_SUM[cluster], point)]
        CS_SUM_SQ[cluster] = [sum(x) for x in zip(CS_SUM_SQ[cluster], [x ** 2 for x in point])]


    #STEP 12
    combine_continue = True
    while combine_continue:
        #print("new_list")
        combine_continue = False
        merge_list = combinations(CS_N.keys(),2)

        for cluster1, cluster2 in merge_list:
            #print(cluster1,cluster2)
            if inter_cluster_maha_distance_CS(cluster1, cluster2) < threshold:
                merge_CS_cluster(cluster1, cluster2)
                combine_continue = True
                break

    RS_set = RS_set_new



    if round==5:
        for C_cluster,_ in CS_N.items():
            dists = []
            min_dist = math.inf
            cluster = -99
            for D_cluster,_ in DS_N.items():
                d = inter_cluster_maha_distance_CS_DS(C_cluster,D_cluster)
                if d < min_dist:
                    cluster=D_cluster
                    min_dist = d
            if min_dist<threshold:
                DS_N[cluster] += CS_N[C_cluster]
                DS_SUM[cluster]=[sum(x) for x in zip(DS_SUM[cluster], CS_SUM[C_cluster])]
                DS_SUM_SQ[cluster]=[sum(x) for x in zip(DS_SUM_SQ[cluster], CS_SUM_SQ[C_cluster])]

    summarize_results_of_round(round)


defaultdict(<class 'int'>, {4: 7643, 7: 7000, 2: 4794, 3: 6179, 6: 6099, 1: 7555, 9: 6135, 0: 6458, 8: 8019, 5: 4564})
defaultdict(<function zero_vector at 0x1a7bba048>, {4: [978561.7243845094, -864129.3929123798, 932660.1178491778, -841067.7083465558, -932405.8763028594, 878818.5391262601, -956067.6776580799, 940398.4401614534, -985980.615347457, -886602.7681569222], 7: [616747.1408328012, -594004.582612324, 547227.2018264135, 582006.7706795882, -595627.4490728762, -630659.3316867265, -575234.4679668192, -490086.5542190606, -558720.3504178697, 630792.0430051644], 2: [762466.7753325464, -790400.8500304486, -796490.6963693078, -743156.8813795155, 796851.9896124286, 752142.4571814474, 810495.2763102924, 799606.143945496, 781112.5565558788, 776322.3437647809], 3: [-1025848.9481025084, -976112.9814755447, -964087.8415906165, -958046.5394059978, -1013765.4367584545, -995333.6330444998, 1044802.4744275184, -1026064.8406714923, 1050085.6479707446, 988138.6981984643], 6: [-513310.1062924455, -

RuntimeError: dictionary changed size during iteration

In [1602]:
DS_Centroids = {}

for cluster, _ in DS_N.items():
    N = DS_N[cluster]
    SUM2 = DS_SUM[cluster]
    centroid = [sum/N for sum in SUM2]
    DS_Centroids[cluster] = centroid

In [1603]:
DS_Centroids

{4: [128.01941571279153,
  -112.96421920271874,
  122.01486400783547,
  -110.00949715690436,
  -121.99346613667595,
  115.035376383252,
  -125.01087352912008,
  122.97390266047401,
  -129.00541564358988,
  -115.96037339411946],
 7: [88.03397177922389,
  -85.01622045586313,
  78.02362385726973,
  83.04258674868119,
  -85.07742345336621,
  -90.00159576455623,
  -82.05711199978707,
  -69.98735237215888,
  -80.03914868244333,
  90.00453844208744],
 2: [159.02823276704058,
  -164.95215951299193,
  -166.01598642426015,
  -154.98951620914573,
  166.0257229285877,
  157.0157000873583,
  168.96246529643935,
  166.8667037617559,
  163.0221442610985,
  161.9684890222442],
 3: [-165.95674555923543,
  -157.98847124511227,
  -156.01280819698246,
  -154.96720205639346,
  -163.98263639833823,
  -161.06244038000824,
  169.0050914616159,
  -166.0117602980612,
  169.9661397414631,
  160.0355862563673],
 6: [-84.0234007336881,
  -72.11967009626412,
  -88.006790434342,
  -84.9653979532689,
  71.02021415457

In [1630]:
assignments_RDD = inputRDD.map(lambda x: (x[0], x[2:])).map(lambda x: (x[0], assign_cluster(x))).collect()

In [1655]:
assignments_RDD = assignments_RDD.collect()

In [1661]:
with open(output, 'w') as f:
    f.write("The intermediate results:\n")
    for r in round_output:
        j = "Round {}: {},{},{},{}\n".format(r[0],r[1],r[2],r[3],r[4])
        f.write(j)

    f.write("\nThe clustering results:\n")

    for i in assignments_RDD:
        j = str(int(i[0])) +  "," + str(int(i[1])) + "\n"
        f.write(j)